In [53]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
import random
import cv2
import os
from math import ceil

ROWS = 64
COLS = 64
CHANNELS=3

In [54]:
train_dir_cats = "../input/cat-and-dog/training_set/training_set/cats/"
test_dir_cats = "../input/cat-and-dog/test_set/test_set/cats/"

train_images_cats=[train_dir_cats+i for i in os.listdir(train_dir_cats)]
test_images_cats=[test_dir_cats+i for i in os.listdir(test_dir_cats)]

train_dir_dogs = "../input/cat-and-dog/training_set/training_set/dogs/"
test_dir_dogs = "../input/cat-and-dog/test_set/test_set/dogs/"

train_images_dogs=[train_dir_dogs+i for i in os.listdir(train_dir_dogs)]
test_images_dogs=[test_dir_dogs+i for i in os.listdir(test_dir_dogs)]

train_images = train_images_cats + train_images_dogs
test_images = test_images_cats + test_images_dogs

train_images.remove('../input/cat-and-dog/training_set/training_set/dogs/_DS_Store')
train_images.remove('../input/cat-and-dog/training_set/training_set/cats/_DS_Store')
test_images.remove('../input/cat-and-dog/test_set/test_set/dogs/_DS_Store')
test_images.remove('../input/cat-and-dog/test_set/test_set/cats/_DS_Store')
random.shuffle(train_images)
random.shuffle(test_images)


In [55]:
print(len(train_images))
print(len(test_images))

8005
2023


In [56]:
"""tn=[]
tt=[]
for i in range(2000):
    tn.append(train_images[i])
for j in range(100):
    tt.append(test_images[j])
train_images=tn
test_images=tt
print(len(train_images))
print(len(test_images))"""

'tn=[]\ntt=[]\nfor i in range(2000):\n    tn.append(train_images[i])\nfor j in range(100):\n    tt.append(test_images[j])\ntrain_images=tn\ntest_images=tt\nprint(len(train_images))\nprint(len(test_images))'

In [57]:
def read_image(file_path):
    if file_path.endswith('.jpg'):
        img = cv2.imread(file_path, 1)
    return cv2.resize(img, (ROWS,COLS), interpolation=cv2.INTER_CUBIC)

In [58]:
def prepare_data(images):

    m = len(images)
    X = np.zeros((m, ROWS, COLS, CHANNELS),dtype=np.uint8)
    y = np.zeros((1, m))
    for i, image_file in enumerate(images):
        #print(image_file)
        X[i,:] = read_image(image_file)
        if 'dog.' in image_file.lower():
            #print("dog")
            y[0, i] = 1
        elif 'cat.' in image_file.lower():
            y[0, i] = 0
            #print("cat")
    return X, y

In [59]:
train_set_x, train_set_y = prepare_data(train_images)
test_set_x, test_set_y = prepare_data(test_images)

In [60]:
train_set_x_flatten = train_set_x.reshape(train_set_x.shape[0], CHANNELS*COLS*ROWS).T
test_set_x_flatten = test_set_x.reshape(test_set_x.shape[0], -1).T
test_set_x=test_set_x_flatten/255
train_set_x=train_set_x_flatten/255


In [61]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

In [62]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    

    w = np.zeros((dim,1))
    b = 0


    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [71]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T, X)+ b)
    
    cost = (-1.0) * np.mean(np.multiply(Y, np.log(A)) + np.multiply(1.0-Y, np.log(1.0 - A)), axis=1)                                 # compute cost
    
    dw = (1/m)*np.dot(X,np.transpose(A-Y))
    db = np.average((A-Y))

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost


In [72]:
w = np.array([[1.],[2.]])
b = 4.
X = np.array([[5., 6., -7.],[8., 9., -10.]])
Y = np.array([[1,0,1]])

grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[4.33333333]
 [6.33333333]]
db = 2.934645119504845e-11
cost = 16.999996678946573


In [73]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        grads, cost = propagate(w,b,X,Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        
        if i % 100 == 0:
            costs.append(cost)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [74]:
params, grads, costs = optimize(w, b, X, Y, num_iterations = 100, learning_rate = 0.009, print_cost = False)

print("w = " + str(params["w"]))
print("b = " + str(params["b"]))
print("dw = " + str(grads["dw"]))
print("db = " + str(grads["db"]))

w = [[-0.49157334]
 [-0.16017651]]
b = 3.948381664135624
dw = [[ 0.03602232]
 [-0.02064108]]
db = -0.01897084202791009


In [65]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T,X)+b)
    
    for i in range(A.shape[1]):
        
        if A[0,i] > 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [66]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.003, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    w, b = initialize_with_zeros(X_train.shape[0])

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost = print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w,b,X_test)
    Y_prediction_train = predict(w,b,X_train)

    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d


In [67]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.003, print_cost = True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.672402
Cost after iteration 200: 0.665198
Cost after iteration 300: 0.660495
Cost after iteration 400: 0.656892
Cost after iteration 500: 0.653895
Cost after iteration 600: 0.651276
Cost after iteration 700: 0.648911
Cost after iteration 800: 0.646730
Cost after iteration 900: 0.644689
Cost after iteration 1000: 0.642759
Cost after iteration 1100: 0.640922
Cost after iteration 1200: 0.639163
Cost after iteration 1300: 0.637474
Cost after iteration 1400: 0.635846
Cost after iteration 1500: 0.634274
Cost after iteration 1600: 0.632752
Cost after iteration 1700: 0.631278
Cost after iteration 1800: 0.629847
Cost after iteration 1900: 0.628457
train accuracy: 65.37164272329794 %
test accuracy: 60.1581809194266 %
